# Import

In [1]:
// %install-swiftpm-flags -c release
// %install '.package(url: "https://github.com/JacopoMangiavacchi/SwiftCoreMLTools.git", from: "0.0.5")' SwiftCoreMLTools
// %install '.package(url: "https://github.com/dduan/Just.git", from: "0.8.0")' Just

In [2]:
import Foundation
import TensorFlow
// import SwiftCoreMLTools
// import Just

# Data Download

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per ten thousand dollars
        - PTRATIO  pupil-teacher ratio by town
        - B        1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
        - LSTAT    % lower status of the population
        - MEDV     Median value of owner-occupied homes in a thousand dollar

    :Missing Attribute Values: None

    :Creator: Harrison, D. and Rubinfeld, D.L.

This is a copy of UCI ML housing dataset.
https://archive.ics.uci.edu/ml/machine-learning-databases/housing/


In [3]:
// if let cts = Just.get(URL(string: "https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data")!).content {
//     try! cts.write(to: URL(fileURLWithPath:"../data/housing.csv"))
// }

# Data Ingestion

In [4]:
let data = try String(contentsOfFile:"../data/housing.csv", encoding: String.Encoding.utf8)
let dataRecords: [[Float]] = data.split(separator: "\n").map{ String($0).split(separator: " ").compactMap{ Float(String($0)) } }

let numRecords = dataRecords.count
let numColumns = dataRecords[0].count

var index = Set<Int>()

while index.count < numRecords {
    index.insert(Int.random(in: 0..<numRecords))
}

let randomDataRecords = index.map{ dataRecords[$0] }

let dataFeatures = randomDataRecords.map{ Array($0[0..<numColumns-1]) }
let dataLabels = randomDataRecords.map{ Array($0[(numColumns-1)...]) }

# Data Transformation

## Split Numerical Categorical Features

In [5]:
let categoricalColumns = [3, 8]
let numericalColumns = [0, 1, 2, 4, 5, 6, 7, 9, 10, 11, 12]
let numCategoricalFeatures = categoricalColumns.count
let numNumericalFeatures = numericalColumns.count
let numLabels = 1

assert(numColumns == numCategoricalFeatures + numNumericalFeatures + 1)

// Get Categorical Features
let allCategoriesValues = dataFeatures.map{ row in categoricalColumns.map{ Int32(row[$0]) } }
                                .reduce(into: Array(repeating: [Int32](), count: 2)){ total, value in
                                    total[0].append(value[0])
                                    total[1].append(value[1]) }
                                .map{ Set($0).sorted() }

let categoricalFeatures = dataFeatures.map{ row in categoricalColumns.map{ Int32(row[$0]) } }

// Get Numerical Features
let numericalFeatures = dataFeatures.map{ row in numericalColumns.map{ row[$0] } }

## Categorize Categorical Features with Ordinal values

In [32]:
var categoricalValues = Array(repeating: Set<Int32>(), count: 2)

for record in categoricalFeatures {
    categoricalValues[0].insert(record[0])
    categoricalValues[1].insert(record[1])
}

let sortedCategoricalValues = [categoricalValues[0].sorted(), categoricalValues[1].sorted()]

let ordinalCategoricalFeatures = categoricalFeatures.map{ [Int32(sortedCategoricalValues[0].firstIndex(of:$0[0])!), 
                                                           Int32(sortedCategoricalValues[1].firstIndex(of:$0[1])!)] }

In [34]:
print(allCategoriesValues[0], allCategoriesValues[1])
print(dataFeatures[9][8])
print(numericalFeatures[9])
print(dataFeatures[9])
print(categoricalFeatures[9])
print(ordinalCategoricalFeatures[9])

[0, 1] [1, 2, 3, 4, 5, 6, 7, 8, 24]
24.0
[9.92485, 0.0, 18.1, 0.74, 6.251, 96.6, 2.198, 666.0, 20.2, 388.52, 16.44]
[9.92485, 0.0, 18.1, 0.0, 0.74, 6.251, 96.6, 2.198, 24.0, 666.0, 20.2, 388.52, 16.44]
[0, 24]
[0, 8]


## Split Train and Test

In [7]:
let trainPercentage:Float = 0.8
let numTrainRecords = Int(ceil(Float(numRecords) * trainPercentage))
let numTestRecords = numRecords - numTrainRecords

func matrixTranspose<T>(_ matrix: [[T]]) -> [[T]] {
    if matrix.isEmpty {return matrix}
    var result = [[T]]()
    for index in 0..<matrix.first!.count {
        result.append(matrix.map{$0[index]})
    }
    return result
}

let xCategoricalAllTrain = matrixTranspose(Array(ordinalCategoricalFeatures[0..<numTrainRecords]))
let xCategoricalAllTest = matrixTranspose(Array(ordinalCategoricalFeatures[numTrainRecords...]))
let xNumericalAllTrain = Array(Array(numericalFeatures[0..<numTrainRecords]).joined())
let xNumericalAllTest = Array(Array(numericalFeatures[numTrainRecords...]).joined())
let yAllTrain = Array(Array(dataLabels[0..<numTrainRecords]).joined())
let yAllTest = Array(Array(dataLabels[numTrainRecords...]).joined())

let XCategoricalTrain = xCategoricalAllTrain.enumerated().map{ (offset, element) in 
    Tensor<Int32>(element).reshaped(to: TensorShape([numTrainRecords, 1]))
}
let XCategoricalTest = xCategoricalAllTest.enumerated().map{ (offset, element) in 
    Tensor<Int32>(element).reshaped(to: TensorShape([numTestRecords, 1]))
}

let XNumericalTrainDeNorm = Tensor<Float>(xNumericalAllTrain).reshaped(to: TensorShape([numTrainRecords, numNumericalFeatures]))
let XNumericalTestDeNorm = Tensor<Float>(xNumericalAllTest).reshaped(to: TensorShape([numTestRecords, numNumericalFeatures]))
let YTrain = Tensor<Float>(yAllTrain).reshaped(to: TensorShape([numTrainRecords, numLabels]))
let YTest = Tensor<Float>(yAllTest).reshaped(to: TensorShape([numTestRecords, numLabels]))

## Normalize Numerical Features

In [8]:
let mean = XNumericalTrainDeNorm.mean(alongAxes: 0)
let std = XNumericalTrainDeNorm.standardDeviation(alongAxes: 0)

print(mean, std)

[[  3.837692,  10.720987,   11.27075, 0.55511284,  6.2830257,    68.3573,   3.760012,    407.558,
   18.485954,   359.1895, 12.7277775]] [[  9.297075,  22.210146,  6.8542833, 0.11656232,  0.7115993,  28.457472,  2.0794964,  169.48473,
    2.119372,  86.309074,  7.3216815]]


In [9]:
let XNumericalTrain = (XNumericalTrainDeNorm - mean)/std
let XNumericalTest = (XNumericalTestDeNorm - mean)/std

In [10]:
print("Training shapes \(XNumericalTrain.shape) \(XCategoricalTrain[0].shape) \(XCategoricalTrain[1].shape) \(YTrain.shape)")
print("Testing shapes  \(XNumericalTest.shape) \(XCategoricalTest[0].shape) \(XCategoricalTest[1].shape) \(YTest.shape)")

Training shapes [405, 11] [405, 1] [405, 1] [405, 1]
Testing shapes  [101, 11] [101, 1] [101, 1] [101, 1]


# Model

In [11]:
struct MultiInputs<N: Differentiable, C>: Differentiable {
  var numerical: N
  
  @noDerivative
  var categorical: C

  @differentiable
  init(numerical: N, categorical: C) {
    self.numerical = numerical
    self.categorical = categorical
  }
}

struct RegressionModel: Module {
    var numericalLayer = Dense<Float>(inputSize: 11, outputSize: 32, activation: relu)
    var embedding1 = Embedding<Float>(vocabularySize: 2, embeddingSize: 2)
    var embedding2 = Embedding<Float>(vocabularySize: 9, embeddingSize: 5)
    var embeddingLayer = Dense<Float>(inputSize: (2 + 5), outputSize: 32, activation: relu)
    var allInputConcatLayer = Dense<Float>(inputSize: (32 + 32), outputSize: 128, activation: relu)
    var hiddenLayer = Dense<Float>(inputSize: 128, outputSize: 32, activation: relu)
    var outputLayer = Dense<Float>(inputSize: 32, outputSize: 1)
    
    @differentiable
    func callAsFunction(_ input: MultiInputs<[Tensor<Float>], [Tensor<Int32>]>) -> Tensor<Float> {
        let numericalInput = numericalLayer(input.numerical[0])
        let embeddingOutput1 = embedding1(input.categorical[0])
        let embeddingOutput1Reshaped = embeddingOutput1.reshaped(to: 
            TensorShape([embeddingOutput1.shape[0], embeddingOutput1.shape[1] * embeddingOutput1.shape[2]]))
        let embeddingOutput2 = embedding2(input.categorical[1])
        let embeddingOutput2Reshaped = embeddingOutput2.reshaped(to: 
            TensorShape([embeddingOutput2.shape[0], embeddingOutput2.shape[1] * embeddingOutput2.shape[2]]))
        let embeddingConcat = Tensor<Float>(concatenating: [embeddingOutput1Reshaped, embeddingOutput2Reshaped], alongAxis: 1)
        let embeddingInput = embeddingLayer(embeddingConcat)
        let allConcat = Tensor<Float>(concatenating: [numericalInput, embeddingInput], alongAxis: 1)
        return allConcat.sequenced(through: allInputConcatLayer, hiddenLayer, outputLayer)
    }
}

var model = RegressionModel()

# Training

In [12]:
let optimizer = RMSProp(for: model, learningRate: 0.001)
Context.local.learningPhase = .training

In [13]:
let epochCount = 500
let batchSize = 32
let numberOfBatch = Int(ceil(Double(numTrainRecords) / Double(batchSize)))
let shuffle = true

func mae(predictions: Tensor<Float>, truths: Tensor<Float>) -> Float {
    return abs(Tensor<Float>(predictions - truths)).mean().scalarized()
}

In [14]:
for epoch in 1...epochCount {
    var epochLoss: Float = 0
    var epochMAE: Float = 0
    var batchCount: Int = 0
    var batchArray = Array(repeating: false, count: numberOfBatch)
    for batch in 0..<numberOfBatch {
        var r = batch
        if shuffle {
            while true {
                r = Int.random(in: 0..<numberOfBatch)
                if !batchArray[r] {
                    batchArray[r] = true
                    break
                }
            }
        }
        
        let batchStart = r * batchSize
        let batchEnd = min(numTrainRecords, batchStart + batchSize)
        let (loss, grad) = model.valueWithGradient { (model: RegressionModel) -> Tensor<Float> in
            let multiInput = MultiInputs(numerical: [XNumericalTrain[batchStart..<batchEnd]],
                                         categorical: [XCategoricalTrain[0][batchStart..<batchEnd],
                                                       XCategoricalTrain[1][batchStart..<batchEnd]])
            let logits = model(multiInput)
            return meanSquaredError(predicted: logits, expected: YTrain[batchStart..<batchEnd])
        }
        optimizer.update(&model, along: grad)
        
        let multiInput = MultiInputs(numerical: [XNumericalTrain[batchStart..<batchEnd]],
                                     categorical: [XCategoricalTrain[0][batchStart..<batchEnd],
                                                   XCategoricalTrain[1][batchStart..<batchEnd]])
        let logits = model(multiInput)
        epochMAE += mae(predictions: logits, truths: YTrain[batchStart..<batchEnd])
        epochLoss += loss.scalarized()
        batchCount += 1
    }
    epochMAE /= Float(batchCount)
    epochLoss /= Float(batchCount)

    print("Epoch \(epoch): MSE: \(epochLoss), MAE: \(epochMAE)")
}

Epoch 1: MSE: 491.70114, MAE: 19.796753
Epoch 2: MSE: 319.2449, MAE: 15.063409
Epoch 3: MSE: 175.89833, MAE: 10.81574
Epoch 4: MSE: 87.37532, MAE: 7.176715
Epoch 5: MSE: 47.04053, MAE: 4.900528
Epoch 6: MSE: 33.370583, MAE: 4.0233173
Epoch 7: MSE: 28.9492, MAE: 3.6320593
Epoch 8: MSE: 27.033323, MAE: 3.4715817
Epoch 9: MSE: 25.723206, MAE: 3.3521864
Epoch 10: MSE: 24.57535, MAE: 3.2877529
Epoch 11: MSE: 23.5631, MAE: 3.1904118
Epoch 12: MSE: 23.107466, MAE: 3.1184504
Epoch 13: MSE: 22.688564, MAE: 3.0788329
Epoch 14: MSE: 21.615004, MAE: 3.018838
Epoch 15: MSE: 21.367666, MAE: 2.9685915
Epoch 16: MSE: 20.618574, MAE: 2.93647
Epoch 17: MSE: 20.335337, MAE: 2.905603
Epoch 18: MSE: 19.758018, MAE: 2.8837278
Epoch 19: MSE: 19.396109, MAE: 2.8424964
Epoch 20: MSE: 19.329313, MAE: 2.8107526
Epoch 21: MSE: 18.523045, MAE: 2.8074877
Epoch 22: MSE: 18.752144, MAE: 2.774503
Epoch 23: MSE: 18.14185, MAE: 2.7374887
Epoch 24: MSE: 17.834278, MAE: 2.7134554
Epoch 25: MSE: 17.616562, MAE: 2.6849096
E

Epoch 203: MSE: 3.7263958, MAE: 1.2213266
Epoch 204: MSE: 3.5707762, MAE: 1.1892135
Epoch 205: MSE: 3.454506, MAE: 1.2166088
Epoch 206: MSE: 3.2851486, MAE: 1.1733152
Epoch 207: MSE: 3.439362, MAE: 1.2310126
Epoch 208: MSE: 3.265523, MAE: 1.268249
Epoch 209: MSE: 3.4684894, MAE: 1.1564978
Epoch 210: MSE: 3.268164, MAE: 1.1488271
Epoch 211: MSE: 4.0976696, MAE: 1.288077
Epoch 212: MSE: 2.9450924, MAE: 1.0932367
Epoch 213: MSE: 3.1447053, MAE: 1.1818864
Epoch 214: MSE: 3.64448, MAE: 1.254548
Epoch 215: MSE: 3.4914217, MAE: 1.1432084
Epoch 216: MSE: 3.2118149, MAE: 1.19139
Epoch 217: MSE: 3.165588, MAE: 1.1283864
Epoch 218: MSE: 3.1297069, MAE: 1.2087779
Epoch 219: MSE: 3.702665, MAE: 1.2124261
Epoch 220: MSE: 3.1566145, MAE: 1.1200536
Epoch 221: MSE: 3.298924, MAE: 1.1428246
Epoch 222: MSE: 3.3472965, MAE: 1.1950616
Epoch 223: MSE: 2.900651, MAE: 1.1368037
Epoch 224: MSE: 3.7760038, MAE: 1.207791
Epoch 225: MSE: 2.934876, MAE: 1.0811876
Epoch 226: MSE: 3.3302736, MAE: 1.1934303
Epoch 227

Epoch 401: MSE: 2.1391423, MAE: 0.9834826
Epoch 402: MSE: 1.9963524, MAE: 0.93260866
Epoch 403: MSE: 1.7422082, MAE: 0.87208134
Epoch 404: MSE: 2.0251353, MAE: 1.0099846
Epoch 405: MSE: 2.4713483, MAE: 0.97791034
Epoch 406: MSE: 1.8526359, MAE: 0.8903376
Epoch 407: MSE: 1.6257775, MAE: 0.7960783
Epoch 408: MSE: 1.8892199, MAE: 1.1056974
Epoch 409: MSE: 2.142658, MAE: 0.99807674
Epoch 410: MSE: 2.1261678, MAE: 0.9529511
Epoch 411: MSE: 2.2038379, MAE: 1.0133382
Epoch 412: MSE: 2.1065068, MAE: 0.94087833
Epoch 413: MSE: 1.9456577, MAE: 0.8830353
Epoch 414: MSE: 1.987342, MAE: 0.94544154
Epoch 415: MSE: 2.1662872, MAE: 0.9680983
Epoch 416: MSE: 2.1216204, MAE: 0.8896621
Epoch 417: MSE: 2.1535184, MAE: 0.9703977
Epoch 418: MSE: 1.6284555, MAE: 0.80591166
Epoch 419: MSE: 2.6078799, MAE: 1.0939535
Epoch 420: MSE: 1.885802, MAE: 0.8518865
Epoch 421: MSE: 1.8285732, MAE: 0.87248856
Epoch 422: MSE: 2.1396022, MAE: 0.97532934
Epoch 423: MSE: 1.8054941, MAE: 0.9314222
Epoch 424: MSE: 1.898833, MA

# Test

In [15]:
Context.local.learningPhase = .inference

let multiInputTest = MultiInputs(numerical: [XNumericalTest],
                                 categorical: [XCategoricalTest[0],
                                               XCategoricalTest[1]])

let prediction = model(multiInputTest)

let predictionMse = meanSquaredError(predicted: prediction, expected: YTest).scalarized()/Float(numTestRecords)
let predictionMae = mae(predictions: prediction, truths: YTest)/Float(numTestRecords)

print("MSE: \(predictionMse), MAE: \(predictionMae)")

MSE: 0.09474914, MAE: 0.023811752
